<a href="https://colab.research.google.com/github/HeatherDriver/MathGraph/blob/main/05_NER_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install seqeval
! pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=5c27c96e143cafbec3ffceeb52cfe8910d9409866502502fcf568d4b17bc7fb1
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.

In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline, AutoConfig, DistilBertForTokenClassification, DistilBertModel, DistilBertConfig, DistilBertPreTrainedModel
from transformers import DataCollatorForTokenClassification, TrainingArguments, Trainer
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.tokenization_utils_base import BatchEncoding
from datasets import Dataset, DatasetDict
# from huggingface_hub import notebook_login
import torch
import torch.nn as nn
from google.colab import drive, userdata
import pickle
import random
import re
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import plotly.express as px
# from seqeval.metrics import classification_report, f1_score, precision_score, recall_score
from seqeval.metrics import classification_report
import evaluate
import pprint

In [3]:
drive.mount("/content/drive")
%cd '/content/drive/MyDrive/Colab Notebooks/Math_Graph/pickle_files'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Math_Graph/pickle_files


In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
metric = evaluate.load('seqeval')

In [6]:
# Define file read function
def read_pickle(dict_file):
  with open(dict_file, 'rb') as file:
    return pickle.load(file)

In [7]:
# Read in dictionary
all_data = read_pickle('all_data_matches.pkl')
all_data_amended = all_data.copy()

In [8]:
random_sample = random.sample(list(all_data_amended.items()), 3)

my_list = []
key, sub_dict = random_sample[0]
my_list.append(sub_dict['tokens'])
my_list.append(sub_dict['baseline_tags'])
my_list.append(sub_dict['input_ids'])
my_list.append(sub_dict['ner_tags'])
print(f"Key: {key}")
print(f"Text: {sub_dict['text']}")

pd.DataFrame(my_list, index=["Tokens", "Baseline_Tags", "Input_ids", "NER_Tags"])

Key: tensor category
Text: A tensor category is a mathematical structure that combines the concepts of category theory and tensor products. It consists of a collection of objects and morphisms, with a binary operation that allows for the "tensoring" of objects and morphisms, satisfying certain coherence conditions. Tensor categories are useful in various fields, including representation theory and quantum physics, as they provide a framework for analyzing symmetries and interactions.


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88
Tokens,[CLS],A,tensor,category,is,a,mathematical,structure,that,combines,the,concepts,of,category,theory,and,tensor,products,.,It,consists,of,a,collection,of,objects,and,m,##or,##phism,##s,",",with,a,binary,operation,that,allows,for,the,"""",tensor,##ing,"""",of,objects,and,m,##or,##phism,##s,",",satisfying,certain,co,##herence,conditions,.,Ten,##sor,categories,are,useful,in,various,fields,",",including,representation,theory,and,quantum,physics,",",as,they,provide,a,framework,for,analyzing,s,##ym,##met,##ries,and,interactions,.,[SEP]
Baseline_Tags,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
Input_ids,101,138,27023,4370,1110,170,9988,2401,1115,14215,1103,8550,1104,4370,2749,1105,27023,2982,119,1135,2923,1104,170,2436,1104,4546,1105,182,1766,19649,1116,117,1114,170,13480,2805,1115,3643,1111,1103,107,27023,1158,107,1104,4546,1105,182,1766,19649,1116,117,18330,2218,1884,21634,2975,119,5157,21484,6788,1132,5616,1107,1672,3872,117,1259,6368,2749,1105,9539,7094,117,1112,1152,2194,170,8297,1111,23389,188,17162,11006,3377,1105,10393,119,102
NER_Tags,O,O,B-FOUNDATIONS-OF-MATHEMATICS,B-FOUNDATIONS-OF-MATHEMATICS,O,O,O,O,O,O,O,O,O,B-FOUNDATIONS-OF-MATHEMATICS,I-FOUNDATIONS-OF-MATHEMATICS,O,O,B-ALGEBRA,O,O,O,O,O,O,O,O,O,B-FOUNDATIONS-OF-MATHEMATICS,O,O,O,O,O,O,B-FOUNDATIONS-OF-MATHEMATICS,B-CALCULUS-AND-ANALYSIS,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-ALGEBRA,I-ALGEBRA,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O


In [9]:
# Finetune definitions
dict_file_name = 'train_definitions.pkl'
train_definitions = read_pickle(dict_file_name)

key, sub_dict = random_sample[0]
# all_data_amended["Kobon Triangle"]['ner_tags'][68] = 'O'
# all_data_amended["Kobon Triangle"]['ner_tags'][69] = 'O'

# train_definitions.update({key: all_data_amended[key]})

dict_file_name = 'train_definitions.pkl'
with open(dict_file_name, 'wb') as file:
  pickle.dump(train_definitions, file)

# Train definitions has the hand-labelled definitions
train_definitions = read_pickle(dict_file_name)
train_definitions.keys()

dict_keys(["Simpson's Paradox", 'Gram-Schmidt Process', 'Cylindrical Parts', 'voter model', 'Skolem-Mahler-Lech Theorem', 'Many-To-One', 'Kobon Triangle'])

## Class distribution by main tag

In [10]:
# Function to count the tag definition per tag-list
def returns_count_per_class(ner_tag_list):
  my_dict = {}
  for tag in ner_tag_list:
    if tag != 'O':
      tag = '-'.join(tag.split('-')[1:])
    if tag not in my_dict:
      my_dict[tag] = 1
    else:
      my_dict[tag] += 1
  sorted_data = dict(sorted(my_dict.items(), key=lambda item: item[1], reverse=True))
  return sorted_data

In [11]:
# Function to summarise main tags
def returns_main_tags(all_data_amended):
  name, count_list, no_tag_list = [], [], []

  for key, sub_dict in all_data_amended.items():
    my_list, my_list_i = [],[]
    to_count = sub_dict['ner_tags']
    count = returns_count_per_class(to_count)

    for i, (k, v) in enumerate(count.items()):
      if k == 'O':
        continue
      else:
        my_list.append(k)
        my_list_i.append(key)
    try:
      count_list.append(my_list[0])
      name.append(my_list_i[0])
    except:
      no_tag_list.append(key)
      continue

  for_df = {'name': name, 'main_concept': count_list}
  df = pd.DataFrame(for_df)
  summary = df[['main_concept']].groupby('main_concept').value_counts().reset_index(drop=False)
  summary = summary.sort_values(by="count", ascending=False).reset_index(drop=True)
  return summary, df, no_tag_list

In [12]:
summary, label_df, no_tag_list = returns_main_tags(all_data_amended)

for tag in no_tag_list:
  try:
    del all_data_amended[tag]
  except:
    continue

assert len(all_data_amended) == label_df.shape[0], 'Error'

In [13]:
fig = px.bar(summary, x='main_concept', y='count', hover_data=['main_concept', 'count'], text='count', color='count', height=500, color_continuous_scale='Viridis', title="Distribution by primary classification label")
fig.update_coloraxes(showscale=False)
fig.show()

### Merging labels together

In [14]:
# Condense matrix classifications to single classification = "MATRICES"

b_tags = ["B-MATRIX-DECOMPOSITION", "B-MATRIX-EIGENVALUES", "B-MATRIX-GROUPS", "B-MATRIX-INVERSION", "B-MATRIX-NORMS", "B-MATRIX-OPERATIONS", "B-MATRIX-PROPERTIES", "B-MATRIX-TYPES", "B-INTEGER-MATRICES"]
i_tags = ["I-MATRIX-DECOMPOSITION","I-MATRIX-EIGENVALUES","I-MATRIX-GROUPS","I-MATRIX-INVERSION","I-MATRIX-NORMS","I-MATRIX-OPERATIONS","I-MATRIX-PROPERTIES","I-MATRIX-TYPES", "I-INTEGER-MATRICES"]
lie_b_tags = ["B-LIE-GROUPS", "B-LIE-THEORY"]
lie_i_tags = ["I-LIE-GROUPS", "I-LIE-THEORY"]

for i, (key, sub_dict) in enumerate(all_data_amended.items()):
  ner_tags = sub_dict['ner_tags']
  for ner_tag in ner_tags:
    if ner_tag in b_tags:
      ner_tags[ner_tags.index(ner_tag)] = "B-MATRICES"
    elif ner_tag in i_tags:
      ner_tags[ner_tags.index(ner_tag)] = "I-MATRICES"
    elif ner_tag in lie_b_tags:
      ner_tags[ner_tags.index(ner_tag)] = "B-LIE-ALGEBRA"
    elif ner_tag in lie_i_tags:
      ner_tags[ner_tags.index(ner_tag)] = "I-LIE-ALGEBRA"
  all_data_amended[key]['ner_tags'] = ner_tags

summary, label_df, no_tag_list = returns_main_tags(all_data_amended)
label_df = label_df.reset_index(drop=False)
fig = px.bar(summary, x='main_concept', y='count', hover_data=['main_concept', 'count'], text='count', color='count', height=500, color_continuous_scale='Viridis', title="Distribution by primary classification label")
fig.update_coloraxes(showscale=False)
fig.show()

In [15]:
# Check order of label_df same as all_data_amended (NB for indices for train-test split)
for i, key in enumerate(list(all_data_amended.keys())):
  assert label_df.iloc[i]['name'] == key, 'Error'

label_df.head()

,index,name,main_concept
0,0,ludwig's inversion formula,CALCULUS-AND-ANALYSIS
1,1,quotient,NUMBER-THEORY
2,2,survivorship curve,APPLIED-MATHEMATICS
3,3,inadmissible,DISCRETE-MATHEMATICS
4,4,Natural Logarithm of 2,DISCRETE-MATHEMATICS


## Train-test stratified split

In [16]:
X_train_indices, X_test_indices, y_train_indices, y_test_indices = train_test_split(label_df['index'].to_numpy(), label_df['main_concept'].to_numpy(),
                                                                                    test_size=0.3, random_state=42, stratify=label_df['main_concept'].to_numpy())

# Repeat to get validation sub-sample of Train
X_train_indices, X_valid_indices, y_train_indices, y_valid_indices = train_test_split(X_train_indices, y_train_indices, test_size=0.3, random_state=42, stratify=y_train_indices)

In [17]:
# Check the resulting distributions
print("Class distribution in original dataset:")
print(label_df['main_concept'].value_counts(normalize=True))

print("\nClass distribution in train dataset:")
print(pd.Series(y_train_indices).value_counts(normalize=True))

print("\nClass distribution in validation dataset:")
print(pd.Series(y_valid_indices).value_counts(normalize=True))

Class distribution in original dataset:
main_concept
CALCULUS-AND-ANALYSIS          0.215533
NUMBER-THEORY                  0.179963
TOPOLOGY                       0.108640
ALGEBRA                        0.100919
GEOMETRY                       0.092188
DISCRETE-MATHEMATICS           0.073621
PROBABILITY-AND-STATISTICS     0.069301
RECREATIONAL-MATHEMATICS       0.037408
APPLIED-MATHEMATICS            0.033915
MATRICES                       0.033272
FOUNDATIONS-OF-MATHEMATICS     0.031618
ORG                            0.008272
LOC                            0.005239
LINEAR-ALGEBRA                 0.004871
DETERMINANTS                   0.001838
LINEAR-SYSTEMS-OF-EQUATIONS    0.001746
LIE-ALGEBRA                    0.001654
Name: proportion, dtype: float64

Class distribution in train dataset:
CALCULUS-AND-ANALYSIS          0.215532
NUMBER-THEORY                  0.180079
TOPOLOGY                       0.108610
ALGEBRA                        0.100919
GEOMETRY                       0.092

## Developing the Model Parameters

In [18]:
# Massive class imbalance between labels - will modify the loss function to perform a weighted loss based on frequency of classes.
# The weights will just be 1 minus the proportion for the main label.
# Eg for  CALCULUS-AND-ANALYSIS = 1 - 0.215533 = 0.784467

_total_count_dictionary = dict()

for i, (key, sub_dict) in enumerate(all_data_amended.items()):
  my_dict = dict()
  my_list, my_list_i = [],[]
  to_count = sub_dict['ner_tags']
  for tag in to_count:
    if tag not in my_dict:
      my_dict[tag] = 1
    else:
      my_dict[tag] += 1
  for k, v in my_dict.items():
    if k not in _total_count_dictionary:
      _total_count_dictionary[k] = v
    else:
      _total_count_dictionary[k] += v

total_count_dictionary = dict(sorted(_total_count_dictionary.items(), key=lambda item: item[1], reverse=True))

total_count = sum(total_count_dictionary.values())
proportions = {k : v/total_count for k, v in total_count_dictionary.items()}

assert np.round(sum(proportions.values()),0) == 1, 'Error'

weights  = {k : 1 - v for k, v in proportions.items()}
class_weights = torch.tensor([weights[cls] for cls in sorted(proportions.keys())], dtype=torch.float32)

loss_fct = nn.CrossEntropyLoss(weight=class_weights)

In [19]:
# Load DistilBERT

tag_checkpoint = "dslim/distilbert-NER"
tag_tokenizer = AutoTokenizer.from_pretrained(tag_checkpoint, do_lower_case=False)
tag_model = AutoModelForTokenClassification.from_pretrained(tag_checkpoint)

tags = pipeline("ner", model=tag_model, tokenizer=tag_tokenizer)

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/926 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

Device set to use cuda:0


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_collator = DataCollatorForTokenClassification(tokenizer=tag_tokenizer, padding=True, return_tensors="pt")

In [21]:
# Need mapping of NER tags to their indices for model to use

tag_list = []
for key, sub_dict in all_data_amended.items():
  ner_tags = sub_dict['ner_tags']
  for tag in ner_tags:
    if tag not in tag_list:
      tag_list.append(tag)

tag_list = sorted(tag_list, key=lambda x: x.split('-', 1)[1] if '-' in x else '')

index2tag = {idx:tag for idx, tag in enumerate(tag_list)} # This is just a nonsignificant arbitrary mapping of the label to a number for training the model
tag2index = {tag:idx for idx, tag in enumerate(tag_list)} # To lookup indices from tags

index2tag[-100] = 'O'
del index2tag[0]

tag2index['O'] = -100 # O-tags set to -100 so that ignored and subwords are not trained upon
index2tag

{1: 'B-ALGEBRA',
 2: 'I-ALGEBRA',
 3: 'B-APPLIED-MATHEMATICS',
 4: 'I-APPLIED-MATHEMATICS',
 5: 'B-CALCULUS-AND-ANALYSIS',
 6: 'I-CALCULUS-AND-ANALYSIS',
 7: 'B-DETERMINANTS',
 8: 'I-DETERMINANTS',
 9: 'B-DISCRETE-MATHEMATICS',
 10: 'I-DISCRETE-MATHEMATICS',
 11: 'B-FOUNDATIONS-OF-MATHEMATICS',
 12: 'I-FOUNDATIONS-OF-MATHEMATICS',
 13: 'B-GEOMETRY',
 14: 'I-GEOMETRY',
 15: 'B-LIE-ALGEBRA',
 16: 'I-LIE-ALGEBRA',
 17: 'B-LINEAR-ALGEBRA',
 18: 'I-LINEAR-ALGEBRA',
 19: 'B-LINEAR-INDEPENDENCE',
 20: 'B-LINEAR-SYSTEMS-OF-EQUATIONS',
 21: 'I-LINEAR-SYSTEMS-OF-EQUATIONS',
 22: 'B-LOC',
 23: 'I-LOC',
 24: 'B-MATRICES',
 25: 'I-MATRICES',
 26: 'B-NUMBER-THEORY',
 27: 'I-NUMBER-THEORY',
 28: 'B-ORG',
 29: 'I-ORG',
 30: 'B-PERMANENTS',
 31: 'B-PROBABILITY-AND-STATISTICS',
 32: 'I-PROBABILITY-AND-STATISTICS',
 33: 'B-RECREATIONAL-MATHEMATICS',
 34: 'I-RECREATIONAL-MATHEMATICS',
 35: 'B-TOPOLOGY',
 36: 'I-TOPOLOGY',
 -100: 'O'}

In [22]:
# Replace ner_indices (labels) in all_data_amended with the index from index2tag

for key, sub_dict in all_data_amended.items():
  ner_tags = sub_dict['ner_tags']
  my_list = []
  for tag in ner_tags:
    idx = tag2index[tag]
    my_list.append(idx)
  all_data_amended[key]['labels'] = my_list

In [23]:
# Custom model based off of pretrained DistilBERT

class DistilBertForTokenClassification(DistilBertPreTrainedModel):

  def __init__(self, config):
    super().__init__(config)
    self.num_labels = config.num_labels

    # Model body
    self.distilbert = DistilBertModel(config)

    # Classification head
    self.dropout = nn.Dropout(config.dropout)
    self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    # Initialise weights
    self.init_weights()

  def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
    # Remove num_items_in_batch from kwargs if present
    kwargs.pop('num_items_in_batch', None)
    # print(f"input_ids shape: {input_ids.shape}")
    # print(f"attention_mask shape: {attention_mask.shape}")

    outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask, **kwargs)
    sequence_output = self.dropout(outputs[0])
    logits = self.classifier(sequence_output)

    # Loss calc
    loss = None
    if labels is not None:
      # loss_fct = loss_fct
      # print(f"labels shape: {labels.shape}")
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

    final_output = TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions)
    return final_output

In [24]:
# Update the existing config to the settings for custom DistilBERT
config = AutoConfig.from_pretrained(tag_checkpoint, num_labels=len(index2tag), label2id=tag2index, id2label=index2tag,)
config.label2id = tag2index
config.id2label = index2tag
config.num_labels = len(index2tag)
config.max_position_embeddings = 512

# Initialise custom DistilBERT using these changed configurations
tag_model_custom = DistilBertForTokenClassification(config)

In [25]:
# Load the pretrained state_dict
pretrained_model = AutoModelForTokenClassification.from_pretrained(tag_checkpoint)

# Use pretrained state_dict in tag_model_custom
tag_model_custom.distilbert.load_state_dict(pretrained_model.distilbert.state_dict(), strict=False)

tag_model_custom.classifier = nn.Linear(config.hidden_size, config.num_labels)
tag_model_custom.init_weights()

tag_model_custom.to(device)

DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
   

## Checking untrained prediction ability

In [26]:
data_dict = dict()
for key, sub_dict in all_data_amended.items():
  my_dict = {k : [v] for k, v in sub_dict.items()}
  data_dict[key] = BatchEncoding(my_dict)

# Convert dictionary to BatchEncoding
batch_encoded_data = BatchEncoding(data_dict)

assert len(batch_encoded_data.keys()) == len(all_data_amended.keys()), 'Error'

In [27]:
# Check key ordering the same between all_data_amended and batch_encoded_data

my_list = list(batch_encoded_data.keys())
for i, key in enumerate(list(all_data_amended.keys())):
  assert my_list[i] == key, 'Error'

In [28]:
def pads_to_max_dimensions(sub_dict):
  # Model needs input_ids and attention_mask to be the maximum size of 512, function pads remainder out to this size
  if isinstance(sub_dict['input_ids'], list):
    input_ids_tensor = torch.tensor(sub_dict['input_ids'], dtype=torch.long)  # Convert list to tensor
  else:
    input_ids_tensor = sub_dict['input_ids'].clone().detach()

  # Convert attention_mask
  if isinstance(sub_dict['attention_mask'], list):
    attention_mask_tensor = torch.tensor(sub_dict['attention_mask'], dtype=torch.long)  # Convert list to tensor
  else:
    attention_mask_tensor = sub_dict['attention_mask'].clone().detach()

  # Convert labels
  if isinstance(sub_dict['labels'], list):
    labels_mask_tensor = torch.tensor(sub_dict['labels'], dtype=torch.long)  # Convert list to tensor
  else:
    labels_mask_tensor = sub_dict['labels'].clone().detach()

  # Padding lengths
  max_len = config.max_position_embeddings
  input_padding_len = max_len - input_ids_tensor.shape[1]
  attention_padding_len = max_len - attention_mask_tensor.shape[1] #config.max_position_embeddings - attention_mask_tensor.shape[0]
  labels_padding_len = max_len - labels_mask_tensor.shape[1]

  # Both need to be padded out to the number of dimensions (512)
  dim_input_ids = torch.nn.functional.pad(input_ids_tensor, (0, input_padding_len), value=0)
  dim_attention_mask = torch.nn.functional.pad(attention_mask_tensor, (0, attention_padding_len), value=0)
  dim_labels = torch.nn.functional.pad(labels_mask_tensor, (0, labels_padding_len), value=-100)

  sub_dict['input_ids'] = dim_input_ids
  sub_dict['attention_mask'] = dim_attention_mask
  sub_dict['labels'] = dim_labels

  return sub_dict

In [29]:
# Implementing above fn
for key, sub_dict in batch_encoded_data.items():
  batch_encoded_data[key] = pads_to_max_dimensions(sub_dict)

## Model functions

In [30]:
idx2key = {idx: key for idx, key in enumerate(batch_encoded_data.keys())}

# Function to create Hugging Face dataset split
def create_dataset_split(indices, batch_encoded_data):

  input_ids = [batch_encoded_data[idx2key[i]]['input_ids'] for i in indices]
  attention_mask = [batch_encoded_data[idx2key[i]]['attention_mask'] for i in indices]
  labels = [batch_encoded_data[idx2key[i]]['labels'] for i in indices]

  dataset_dict = {
      'input_ids': input_ids,
      'attention_mask': attention_mask,
      'labels': labels
  }
  return Dataset.from_dict(dataset_dict)

In [31]:
# Create train, validation and test sets

train_dataset = create_dataset_split(X_train_indices, batch_encoded_data)
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

validation_dataset = create_dataset_split(X_valid_indices, batch_encoded_data)
validation_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

test_dataset = create_dataset_split(X_test_indices, batch_encoded_data)
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Combine into a DatasetDict
dataset = DatasetDict({'train': train_dataset, 'validation': validation_dataset, 'test': test_dataset})

In [53]:
def returns_actual_and_predictions(outputs, labels):
  # Since only training on idxs > -100, fn removes these
  # Format of y_actual_labels is datadict['labels']
  prediction_idx = torch.argmax(outputs, dim=-1)
  prediction_batch_size, prediction_dim = prediction_idx.shape #1, 512

  actual_dim = int(outputs.squeeze().shape[0])

  actual, predicted, _actual, _preds = [], [], [], []
  for i in range(actual_dim):
    actual_idx = outputs.squeeze()[i].item()
    pred_idx = prediction_idx[0][i].item()
    if pred_idx == 0:
      pred_idx = -100
    if actual_idx != -100 and actual_idx > 0:
      _preds.append(index2tag[pred_idx])
      _actual.append(index2tag[actual_idx])
    if actual_idx == 0:
      break

  predicted.append(_preds)
  actual.append(_actual)
  return actual, predicted

def compute_metrics(eval_pred):
  actual_tags, predicted_tags = returns_actual_and_predictions(eval_pred.predictions, eval_pred.label_ids)
  metric_result_dict = metric.compute(predictions=predicted_tags, references=actual_tags)
  f1_score = metric_result_dict['overall_f1']
  return {'f1_score': f1_score}

def model_init():
  return tag_model_custom.to(device)

## Test Model function on small pilot set

In [33]:
sample = train_dataset[0]
inputs = {
    'input_ids': sample['input_ids'].clone().detach(),
    'attention_mask': sample['attention_mask'].clone().detach()}

tag_model_custom.eval()

with torch.no_grad():
  outputs = tag_model_custom.forward(**inputs).logits

y_actual = sample['labels']

actual, predicted = returns_actual_and_predictions(outputs, y_actual)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
# Display in df: across all 512 dimensions as a double-check
input_ids = inputs['input_ids'][0].numpy() #X
tokens = tag_tokenizer.convert_ids_to_tokens(inputs['input_ids'][0].numpy()) #X in human sub-token form
prediction_idx = torch.argmax(outputs, dim=-1)
untrained_model_id = prediction_idx[0].numpy() #Tag predictions in numeric form
y_pred = [index2tag.get(p, 'O') for p in untrained_model_id] #Tag predictions in human word form
attn_mask = inputs['attention_mask'][0].numpy()
actual_idx = sample['labels'].squeeze().numpy() #Actual tag index
actual_ner = [index2tag[i] for i in actual_idx if i!= 0] #Actual tags

pd.DataFrame([input_ids, tokens, untrained_model_id, y_pred, attn_mask, actual_ner, actual_idx],
             index=["Input_ids", "Tokens", "Untrained_ID","Untrained_Tag_Prediction", "Attention_Mask", "Actual_NER_Tags", "Actual_NER_idx"])

In [ ]:
print(classification_report(actual, predicted))

## Model Training

In [54]:
# Fixing input_ids shape in the dataset

def fix_shape(example):
  example['input_ids'] = example['input_ids'].squeeze(0).tolist()
  example['attention_mask'] = example['attention_mask'].squeeze(0).tolist()
  example['labels'] = example['labels'].squeeze(0).tolist()
  return example

dataset['train'] = dataset['train'].map(fix_shape)
dataset['validation'] = dataset['validation'].map(fix_shape)
dataset['test'] = dataset['test'].map(fix_shape)

Map:   0%|          | 0/5331 [00:00<?, ? examples/s]

Map:   0%|          | 0/2285 [00:00<?, ? examples/s]

Map:   0%|          | 0/3264 [00:00<?, ? examples/s]

In [63]:
# Update function so that runs on Batch
def returns_actual_and_predictions(outputs, labels):
  # Convert outputs to a PyTorch tensor if it's a NumPy array
  outputs = torch.from_numpy(outputs) if isinstance(outputs, np.ndarray) else outputs

  prediction_idx = torch.argmax(outputs, dim=-1)
  prediction_batch_size, prediction_dim = prediction_idx.shape
  actual, predicted = [], []

  for i in range(prediction_batch_size):
    _labels, _preds = [], []
    for j in range(prediction_dim):
      label = labels[i][j].item()
      # ignore label of -100
      if label != -100:
        _labels.append(index2tag[label])
        _preds.append(index2tag[prediction_idx[i][j].item()])
    actual.append(_labels)
    predicted.append(_preds)
  return actual, predicted

def compute_metrics(eval_pred):
  actual_tags, predicted_tags = returns_actual_and_predictions(eval_pred.predictions, eval_pred.label_ids)
  metric_result_dict = metric.compute(predictions=predicted_tags, references=actual_tags)
  f1_score = metric_result_dict['overall_f1']
  return {'f1_score': f1_score}

In [64]:
num_epochs = 3
batch_size = 5
learning_rate = 3e-5
# logging_steps = len(dataset["train"]) // batch_size
model_name = f"distilbert-NER-Math-finetuned"

model_arguments = TrainingArguments(
    output_dir=model_name,
    log_level="error",
    logging_strategy="steps",
    logging_steps=50,
    weight_decay=0.01,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    eval_strategy="epoch",
    disable_tqdm=False,
    save_steps=1000000,
    remove_unused_columns=False,
    push_to_hub=True,
    no_cuda=False
)

data_collator = DataCollatorForTokenClassification(
    tokenizer=tag_tokenizer,
    return_tensors="pt",
    padding='max_length',
    label_pad_token_id=-100  # Padding token for labels
)

In [65]:
trainer = Trainer(
    model_init=model_init, args=model_arguments, data_collator=data_collator, compute_metrics=compute_metrics,
    tokenizer=tag_tokenizer, train_dataset=dataset['train'], eval_dataset=dataset['validation']
    )

<ipython-input-65-537117c56c34>:1: FutureWarning:

`tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.



In [66]:
torch.cuda.empty_cache()

In [67]:
# Run on a single batch of 5
train_dataloader = trainer.get_train_dataloader()

for batch in train_dataloader:
  # Check the batch structure
  print(f"input_ids shape {batch['input_ids'].shape}")
  print(f"attention_mask shape {batch['attention_mask'].shape}")
  print(f"labels shape {batch['labels'].shape}\n")
  break

input_ids shape torch.Size([5, 512])
attention_mask shape torch.Size([5, 512])
labels shape torch.Size([5, 512])



In [68]:
# If cuda:
tag_model_custom.train()  # Set model to training mode

# Move class_weights to the same device as the model
class_weights = class_weights.to(device)
input_ids = batch['input_ids'].to(device)
attention_mask = batch['attention_mask'].to(device)
labels = batch['labels'].to(device)
tag_model_custom = tag_model_custom.to(device)
loss_fct = loss_fct.to(device)

print(f"Model device: {tag_model_custom.device}")
print(f"Class weights device: {class_weights.device}")
print(f"Input IDs device: {input_ids.device}")
print(f"Attention mask device: {attention_mask.device}")
print(f"Labels device: {labels.device}")

# Ensure inputs are on the same device
outputs = tag_model_custom(
    input_ids=batch['input_ids'].to(device),
    attention_mask=batch['attention_mask'].to(device),
    labels=batch['labels'].to(device)
)
loss = outputs.loss
print(f"logits shape: {outputs.logits.shape}")
print(f"loss: {loss}")
loss.backward()

optimizer = torch.optim.AdamW(tag_model_custom.parameters(), lr=learning_rate)
optimizer.step()
optimizer.zero_grad()

Model device: cuda:0
Class weights device: cuda:0
Input IDs device: cuda:0
Attention mask device: cuda:0
Labels device: cuda:0
logits shape: torch.Size([5, 512, 37])
loss: 0.00649065338075161


In [46]:
# If no cuda - uncomment and run
# tag_model_custom.train()  # Set model to training mode

# outputs = tag_model_custom(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])
# loss = outputs.loss
# print(f"logits shape: {outputs.logits.shape}")
# print(f"loss: {loss}")
# loss.backward()

# optimizer = torch.optim.AdamW(tag_model_custom.parameters(), lr=learning_rate)
# optimizer.step()
# optimizer.zero_grad()

In [69]:
# Check functioning of trainer directly

batch = next(iter(train_dataloader))  # Get a single batch
outputs = trainer.training_step(trainer.model, batch)

print(outputs)  # Loss for the batch

tensor(0.1349, device='cuda:0')


In [70]:
# forward propagation, loss computation, backpropagation, and optimization all handled by Hugging Face Trainer
trainer.train()
trainer.push_to_hub(commit_message="NER Training")

Epoch,Training Loss,Validation Loss,F1 Score
1,0.042900,0.439037,0.895987
2,0.103000,0.376712,0.904194
3,0.023800,0.372616,0.915125


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



events.out.tfevents.1735819471.705a1d9f60bb.560.3:   0%|          | 0.00/22.0k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/HeathStar/distilbert-NER-Math-finetuned/commit/a749f80d7c0d82f49604be8db35f192896808094', commit_message='NER Training', commit_description='', oid='a749f80d7c0d82f49604be8db35f192896808094', pr_url=None, repo_url=RepoUrl('https://huggingface.co/HeathStar/distilbert-NER-Math-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='HeathStar/distilbert-NER-Math-finetuned'), pr_revision=None, pr_num=None)

In [ ]:
# %cd '/content/drive/MyDrive/Colab Notebooks/Math_Graph'
# tag_model_custom.save_pretrained("math_ner_model")
# tag_tokenizer.save_pretrained("math_ner_model")